# Init

## Load libs

In [1]:
import numpy as np
import plotly.graph_objs as go
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
from itertools import chain
from datetime import datetime
from torch.utils.data import ConcatDataset, DataLoader, random_split

from pytorch_lightning import seed_everything, Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    root_mean_squared_error,
    r2_score
)

from shaft_force_sensing import ForceSensingDataset
from shaft_force_sensing.models import LitLTC
from shaft_force_sensing.evaluation import (
    tb_to_numpy,
    add_norm,
    array_bais,
    array_medfilt,
)

%load_ext autoreload
%autoreload 2

## Set hyperparameters

In [2]:
seed_everything(42)
max_epochs = 30
batch_size = 256
learning_rate = 1e-4
hidden_size = 128
num_layers = 3
num_heads = 8

[rank: 0] Seed set to 42


In [3]:
axes = ['F_x', 'F_y', 'F_z', 'Norm']

# Evaluation

In [9]:
run_dir = Path("/scratch/pioneer/users/sxk2514/shaft_force_sensing/logs/20260216_225606")
save_dir = run_dir / "LitLTC" / "version_0"   # training logs
zero_shot_dir = run_dir / "test" / "Free"     # single dataset
save_dir = run_dir / "test"                   # all datasets


In [11]:
def get_checkpoint_path(run_dir: Path) -> Path:
    ckpts = sorted(run_dir.glob("best*.ckpt"))
    if not ckpts:
        raise FileNotFoundError(f"No checkpoint found in {run_dir}")
    return ckpts[-1]   # latest / best
ckpt_path = get_checkpoint_path(run_dir)
print("Checkpoint:", ckpt_path)


Checkpoint: /scratch/pioneer/users/sxk2514/shaft_force_sensing/logs/20260216_225606/best-epoch=24-loss=0.0730.ckpt


In [12]:
model = LitLTC.load_from_checkpoint(
    ckpt_path,
    map_location="cpu"
)


In [13]:

golbal_scaler = StandardScaler()
golbal_scaler.mean_ = model.data_mean.numpy(force=True)
golbal_scaler.scale_ = model.data_std.numpy(force=True)

del model

## Single set

In [16]:
gt, pred = tb_to_numpy(zero_shot_dir)
zero_shot_dir.stem

'Free'

Denormlization

In [17]:
gt = golbal_scaler.inverse_transform(gt)
pred = golbal_scaler.inverse_transform(pred)

Smooth

In [18]:
pred = array_medfilt(pred, kernel_size=71)

Zero offset

In [19]:
pred = array_bais(pred, 50)

Add norm

In [20]:
gt = add_norm(gt)
pred = add_norm(pred)

Time plot

In [21]:
d = gt.shape[1]
fig = make_subplots(rows=d, cols=1, shared_xaxes=True, subplot_titles=axes)

for i, name in enumerate(axes, start=1):
    fig.add_trace(go.Scatter(y=gt[:, i-1], mode="lines", name=f"{name} (gt)"), row=i, col=1)
    fig.add_trace(go.Scatter(y=pred[:, i-1], mode="lines", name=f"{name} (pred)"), row=i, col=1)

fig.update_layout(height=250 * d, title="Ground Truth vs Prediction", showlegend=True)
fig.show()

## Loop all sets

In [22]:
from pathlib import Path
from tqdm import tqdm


data = dict()

for path in tqdm([p for p in save_dir.iterdir() if p.is_dir()]):
    group = path.stem

    # Load data
    gt, pred = tb_to_numpy(path)

    # Post-processing
    gt = golbal_scaler.inverse_transform(gt)
    pred = golbal_scaler.inverse_transform(pred)

    pred = array_medfilt(pred, kernel_size=71)
    pred = array_bais(pred, 50)

    gt = add_norm(gt)
    pred = add_norm(pred)

    data[group] = (gt, pred)


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:33<00:00, 11.15s/it]


In [23]:
gt_all = np.concatenate([data[group][0] for group in data], axis=0)
pred_all = np.concatenate([data[group][1] for group in data], axis=0)
data['All'] = (gt_all, pred_all)

In [24]:
for group, (gt, pred) in data.items():
    # Metrics
    gt_min = np.min(gt, axis=0)
    gt_max = np.max(gt, axis=0)
    gt_range = gt_max - gt_min
    rmse = root_mean_squared_error(gt, pred, multioutput='raw_values')
    nrmse = rmse / gt_range
    r2_scores = r2_score(gt, pred, multioutput='raw_values')

    # Logging
    with open(save_dir / f"metrics.txt", "a") as f:
        print(f"Group: {group}", file=f)
        for i, name in enumerate(axes):
            print(
                f"{name}: \
                Range={gt_range[i]:.4f}, \
                RMSE={rmse[i]:.4f}, \
                NRMSE={nrmse[i]*100:.2f}%, \
                R2={r2_scores[i]*100:.2f}",
                file=f)
        print("-" * 10, file=f)